In [1]:
from osman import Osman, OsmanConfig
import pandas as pd

In [3]:
OS_HOST_URL="http://localhost:9200"
TEMPLATE_NAME = 'semantic_search'
INDEX_NAME = "bible"

In [4]:
os_man = Osman(OsmanConfig(host_url=OS_HOST_URL))

In [5]:
cluster_settings = {
    "transient": {
        "plugins.ml_commons.model_access_control_enabled": True,
        "plugins.ml_commons.allow_registering_model_via_url": True,
        "plugins.ml_commons.only_run_on_ml_node": False,
    },
    "persistent": {
        "plugins": {
            "ml_commons": {
                "only_run_on_ml_node": "false",
                "model_access_control_enabled": "true",
                "native_memory_threshold": "99"
            }
        }
    }
}

In [6]:
os_man.update_cluster_settings(cluster_settings)

{'acknowledged': True,
 'persistent': {'plugins': {'ml_commons': {'only_run_on_ml_node': 'false',
    'model_access_control_enabled': 'true',
    'native_memory_threshold': '99'}}},
 'transient': {'plugins': {'ml_commons': {'only_run_on_ml_node': 'false',
    'allow_registering_model_via_url': 'true',
    'model_access_control_enabled': 'true'}}}}

In [7]:
os_man.client.cluster.get_settings()

{'persistent': {'plugins': {'ml_commons': {'only_run_on_ml_node': 'false',
    'model_access_control_enabled': 'true',
    'native_memory_threshold': '99'},
   'index_state_management': {'template_migration': {'control': '-1'}}}},
 'transient': {'plugins': {'ml_commons': {'only_run_on_ml_node': 'false',
    'allow_registering_model_via_url': 'true',
    'model_access_control_enabled': 'true'}}}}

In [8]:
model_payload = {
    "name": "example-model",
    "version": "1.0.0",
    "model_format": "TORCH_SCRIPT"
}
os_man.send_post_request(
    "/_plugins/_ml/models/_register",
    model_payload
)

{'task_id': 'KXEt_pABsga7C9sF5WAN', 'status': 'CREATED'}

In [9]:
task_id = "12xrxZABMGFYeblGJ_ln"
endpoint = f"/_plugins/_ml/tasks/{task_id}"

os_man.send_get_request(endpoint)

{'model_id': '2GxrxZABMGFYeblGLfke',
 'task_type': 'REGISTER_MODEL',
 'function_name': 'TEXT_EMBEDDING',
 'state': 'COMPLETED',
 'worker_node': ['nKXVxBBkT0icEj3xqD4f5A'],
 'create_time': 1721299052391,
 'last_update_time': 1721299086189,
 'is_async': True}

In [10]:
model_id = "2GxrxZABMGFYeblGLfke"
endpoint = f"/_plugins/_ml/models/{model_id}/_deploy"
payload = {}
os_man.send_post_request(endpoint, payload)


{'task_id': 'KnEt_pABsga7C9sF7GAt',
 'task_type': 'DEPLOY_MODEL',
 'status': 'CREATED'}

In [11]:
pipeline_id = "nlp-ingest-pipeline"
endpoint = f"/_ingest/pipeline/{pipeline_id}"

# Define the payload for the ingest pipeline
pipeline_payload = {
    "description": "An NLP ingest pipeline",
    "processors": [
        {
            "text_embedding": {
                "model_id": "2GxrxZABMGFYeblGLfke",
                "field_map": {
                    "content": "passage_embedding"
                }
            }
        }
    ]
}

os_man.send_put_request(endpoint, pipeline_payload)

{'acknowledged': True}

In [12]:
index_name = "bible-nlp"
endpoint = f"/{index_name}"

index_payload = {
    "settings": {
        "index.knn": True,
        "default_pipeline": "nlp-ingest-pipeline"
    },
    "mappings": {
        "properties": {
            "id": {
                "type": "integer"
            },
            "verse_id": {
                "type": "keyword"
            },
            "passage_embedding": {
                "type": "knn_vector",
                "dimension": 768,
                "method": {
                    "engine": "lucene",
                    "space_type": "l2",
                    "name": "hnsw",
                    "parameters": {}
                }
            },
            "content": {
                "type": "text"
            }
        }
    }
}

os_man.send_put_request(endpoint, index_payload)

In [13]:
TEMPLATE_NAME = 'semantic_search'
source = {
    "_source": {
        "excludes": ["passage_embedding"]
    },
    "query": {
        "bool": {
            "should": [
                {
                    "fuzzy": {
                        "content": {  
                            "value": "{{query_text}}",
                            "fuzziness": 2,  
                            "prefix_length": 1  
                        }
                    }
                }
            ],
            "minimum_should_match": 1  
        }
    }
}

In [63]:
source = {
    "size": 500,
    "query": {
        "neural": {
            "passage_embedding": {
                "query_text": "{{query_text}}",
                "model_id": "{{model_id}}",
                "k": "{{k}}"
                }
            }
        }
    }

In [65]:
TEMPLATE_NAME = 'semantic_search2'
INDEX_NAME = "bible-nlp"

params = {
    "query_text": "christ",
    "model_id": "2GxrxZABMGFYeblGLfke",
    "k": 5
}

os_man.upload_search_template(
    source=source, 
    name=TEMPLATE_NAME, 
    index=INDEX_NAME, 
    params=params
)

{'acknowledged': True,
 'differences': {'size': 500,
  'query': {'neural': {'passage_embedding': {'query_text': '{{query_text}}',
     'model_id': '{{model_id}}',
     'k': '{{k}}'}}}}}

In [38]:
source = {
    "_source": {
        "excludes": ["passage_embedding"]
    },
    "query": {
        "bool": {
            "should": [
                {
                    "fuzzy": {
                        "content": {  
                            "value": "{{query_text}}",
                            "fuzziness": 2,  
                            "prefix_length": 1  
                        }
                    }
                }
            ],
            "minimum_should_match": 1  
        }
    }
}

TEMPLATE_NAME = 'semantic_search2'
INDEX_NAME = "bible-nlp"

params = {
    "query_text": "christ",
    "model_id": "2GxrxZABMGFYeblGLfke",
    "k": 5
}

os_man.upload_search_template(
    source=source, 
    name=TEMPLATE_NAME, 
    index=INDEX_NAME, 
    params=params
)

{'acknowledged': False}

In [39]:
from opensearchpy import OpenSearch

# Connection settings
host = 'localhost'
port = 9200

# Connect to OpenSearch
client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    use_ssl=False,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False
)

In [58]:
# Define the template parameters
template_params = {
    "id": "semantic_search2",  # The name of the template you uploaded
    "params": {
        "query_text": "bla"  
    }
}

# Execute the search using the search template
response = client.search_template(
    body=template_params,
    index="bible-nlp" 
)

# Print the response
print(response)


{'took': 33, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1421, 'relation': 'eq'}, 'max_score': 1.3318306, 'hits': [{'_index': 'bible-nlp', '_id': '78057', '_score': 1.3318306, '_source': {'verse_id': '78:57', 'id': 78057, 'content': 'But turned back, and dealt unfaithfully like their fathers: they\nwere turned aside like a deceitful bow.'}}, {'_index': 'bible-nlp', '_id': '9061', '_score': 1.1733632, '_source': {'verse_id': '9:61', 'id': 9061, 'content': 'And another also said, Lord, I will follow thee; but let me first\ngo bid them farewell, which are at home at my house.'}}, {'_index': 'bible-nlp', '_id': '33006', '_score': 1.0684036, '_source': {'verse_id': '33:6', 'id': 33006, 'content': "But if the watchman see the sword come, and blow not the trumpet,\nand the people be not warned; if the sword come, and take any person\nfrom among them, he is taken away in his iniquity; but his blood will\nI require at the 

In [60]:
# Extracting the data from the response
hits = response['hits']['hits']
data = [
    {
        "Index": hit['_index'],
        "Score": hit['_score'],
        "Verse ID": hit['_source']['verse_id'],
        "Content": hit['_source']['content'].replace('\n', ' ')  # Replacing newline characters for better display
    }
    for hit in hits
]

# Creating a DataFrame
df = pd.DataFrame(data)

In [61]:
df

,Index,Score,Verse ID,Content
0,bible-nlp,1.331831,78:57,"But turned back, and dealt unfaithfully like t..."
1,bible-nlp,1.173363,9:61,"And another also said, Lord, I will follow the..."
2,bible-nlp,1.068404,33:6,"But if the watchman see the sword come, and bl..."
3,bible-nlp,1.064771,44:32,For thy servant became surety for the lad unto...
4,bible-nlp,1.002893,22:36,"Then said he unto them, But now, he that hath ..."
5,bible-nlp,0.947813,3:5,"He that overcometh, the same shall be clothed ..."
6,bible-nlp,0.934975,55:10,"For as the rain cometh down, and the snow from..."
7,bible-nlp,0.922481,2:14,"But I have a few things against thee, because ..."
8,bible-nlp,0.912806,115:5,"They have mouths, but they speak not: eyes hav..."
9,bible-nlp,0.912806,115:6,"They have ears, but they hear not: noses have ..."
